In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Many of the functions in utils.py use variables defined within FDS.ipynb,
#  so those function can only be used from within that notebook by running the following:
#  %run -i code/utils.py

import pandas as pd
import numpy as np
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import os.path

from PIL import Image, ImageFont, ImageDraw
from mpl_toolkits import mplot3d
from matplotlib import cm
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from os import path

# Funciones

In [ ]:
def draw_rectangle(image_in,xlb,xub,ylb,yub,image_out,color):
    # Drawing a rectangle in a picture
    dpi = 80
    img = matplotlib.image.imread(image_in)
    height, width, nbands = img.shape
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)
    
    figure = plt.figure(figsize=figsize)
    ax = figure.add_axes([0, 0, 1, 1])
    # Hide spines, ticks, etc.
    ax.axis('off')
    rect = matplotlib.patches.Rectangle((xlb,height-yub),xub-xlb,yub-ylb, edgecolor=color, facecolor="none",linewidth=2.5)   
    ax.imshow(img)
    ax.add_patch(rect)
    figure.savefig(image_out,dpi=dpi,transparent=True)
    plt.cla()
    plt.close()
    return 0

def crop_rectangle(image_in,xlb,xub,ylb,yub,image_out):
    # Cropping a rectangle from a picture
    im  = Image.open(image_in)
    box = (xlb,im.size[1]-yub,xub,im.size[1]-ylb)
    im.crop(box).save(image_out)

def change_color(image_in,source_color_rgb,destination_color_rgb,image_out):
     
    im = Image.open(image_in)
    im = im.convert('RGBA')

    data = np.array(im)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability

    # Replace white with white... (leaves alpha values alone...)
    obs_areas = (red == source_color_rgb[0])  & (green == source_color_rgb[1]) & (blue == source_color_rgb[2])
    data[..., :-1][obs_areas.T] = (destination_color_rgb[0], destination_color_rgb[1], destination_color_rgb[2]) # Transpose back needed

    im2 = Image.fromarray(data)
    im2.save(image_out)
    
def draw_vector(image_in,X1,X2,image_out):
     
    im = Image.open(image_in)
    image_editable = ImageDraw.Draw(im)
    image_editable.line((X1[0],im.size[1] - X1[1], X2[0], im.size[1] - X2[1]), fill=(0,0,255,255), width=3)
   
    im.save(image_out)

def spread(A,B,C):
    # ta: time when the fire reachs the point A
    # tb: time when the fire reachs the point B
    # tc: time when the fire reachs the point C
    
    # A: coordinates of the point A
    # B: coordinates of the point B
    # C: coordinates of the point C
    
    # Calculating the Angle in A
    ## Vectorizing the points
    vA = np.array(A)
    vB = np.array(B)
    vC = np.array(C)
    
    vAB = vB-vA
    vAC = vC-vA
    
    norm_AB = np.linalg.norm(vAB)
    norm_AC = np.linalg.norm(vAC)
    unit_vAB = vAB / norm_AB
    unit_vAC = vAC / norm_AC
    
    dot_product = np.dot(unit_vAB, unit_vAC)
    angleA = np.arccos(dot_product)
    theta_d = angleA*(180/math.pi) 
    
    return theta_d

def draw_arc(image_in,C,r,angle_initial,angle_final,image_out):
     
    im = Image.open(image_in)
    image_editable = ImageDraw.Draw(im)
    image_editable.arc([(C[0]-r,im.size[1]-(C[1]+r)),(C[0]+r,im.size[1]-(C[1]-r))],angle_initial,angle_final,fill="black")
   
    im.save(image_out)

# Selecting the new region

In [ ]:
camino = '/mnt/d/Dropbox/0Documents/Dr_Taufer_Lab/experiments/Article_experiments_2/frames'

xlb      =  480
xub      =  1200
ylb      =  40    
yub      =  980

draw_rectangle(f'{camino}/Frames_Nist/Gatlinburg_2m_terrain_trees_0002.png',xlb,xub,ylb,yub,f'{camino}/Frames_Cuted/test.png',[255/255,153/255,5/255])

In [ ]:
xlb = 246
xub = 1146
ylb = 20 
yub = 920

X1 = [430, 255]
X2 = [530, 255]
X3 = [48, 643]

Theta = spread(X1,X2,X3)

for i in range(182):
    crop_rectangle(f'{camino}/Region1/Region1_'+f'{i}'.zfill(4)+'.png',xlb,xub,ylb,yub,f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png')
    change_color(f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png',[98,94,38],[255,255,255],f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png')
    
    draw_vector(f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png',X1,X2,f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png')
    draw_vector(f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png',X1,X3,f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png')
    
    for r in range(10,600,10):
        draw_arc(f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png',X1,r,(360-Theta)-(160*360/(4*math.pi*r)),(360-Theta)+(160*360/(4*math.pi*r)),f'{camino}/Region1_Cuted/Region1_'+f'{i}'.zfill(4)+'.png')

In [ ]:
Theta = spread(X1,X2,X3)
Theta